# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы изучите методы работы с категориальными переменными

In [18]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split



%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(2 балла)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [19]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [42]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

np.random.seed(910)

# Создание сбалансированной выборки
def create_balanced_dataset(data, target, samples_per_class=100000):
    class_1_idx = np.where(target == 1)[0]
    class_0_idx = np.where(target == 0)[0]
    
    selected_1 = np.random.choice(class_1_idx, samples_per_class, replace=True)
    selected_0 = np.random.choice(class_0_idx, samples_per_class, replace=True)
    
    combined_data = pd.concat([data.iloc[selected_1], data.iloc[selected_0]])
    combined_target = np.hstack([target[selected_1], target[selected_0]])
    
    return combined_data, combined_target

data, target = create_balanced_dataset(data, target)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalize_features(df):
    standard_scaler = StandardScaler()
    normalized_array = standard_scaler.fit_transform(df)
    return pd.DataFrame(normalized_array, columns=df.columns)

scaled_features = normalize_features(data)
scaled_features.describe()
data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,1.366550,4.483870,0.429490,0.502265,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.674421,2.739255,0.496689,1.501934,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,7.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию, найдтие оптимум. Сделайте предсказание на тестовой части выборки. Замерьте качество.

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
def getScore(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return [accuracy, precision, recall, f1]
def printScore(score):
    print(f"Accuracy: {score[0]:.4f}")
    print(f"Precesion: {score[1]:.4f}")
    print(f"Recall: {score[2]:.4f}")
    print(f"F1 Score: {score[3]:.4f}")

X_train, X_test, y_train, y_test = train_test_split(normalized_data, target, test_size=0.7)

printScore(getScore(X_train, y_train, X_test, y_test))


Accuracy: 0.5898
Precesion: 0.5965
Recall: 0.5526
F1 Score: 0.5737


__Выводы__ в свободной форме:

Модель LogisticRegression (логистической регрессии) уже из коробки предоставляет нам довольно неплохие результаты. Остается правильно отрегулировать и правильно настроить, чтобы повысить эффективность модели.

__Задание 2.__ Изучение влияния регуляризатора на процесс обучения

__(2 балла)__

Проверьте на практике, как влияет регуляризатор на процесс обучения (убывание функции потерь на обучающей и отложенной выборках). Чтобы считать функцию потерь на отложенной выборке после каждой итерации, запускайте процесс обучения логистической регрессии с параметром $max\_iter=1$ и $w^{(0)}$, полученным на предыдущей итерации. Постройте два графика: на одном из них логистическая регрессия с коэффициентом регуляризации, равным 0, а на другом с некоторым разумным значением. На каждом графике одновременно должна быть и функция потерь для обучающей, и для тестовой выборки. Не забудьте сделать одинаковыми оси обоих графиков. Какие выводы вы можете сделать?

In [17]:
# Your code here
# ...fds

__Выводы:__

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [31]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = [c for c in normalized_data.columns if c.endswith('_cat')]

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_features = encoder.fit_transform(normalized_data[categorical_features])

data_one_hot = normalized_data.drop(categorical_features, axis=1)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

data_one_hot.reset_index(inplace=True, drop=True)
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)

X_train, X_test, y_train, y_test = train_test_split(normalize(data_one_hot), target, test_size=0.5)
data_one_hot.describe()



,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,...,ps_car_11_cat_0.9698844814339967,ps_car_11_cat_0.9999078260160954,ps_car_11_cat_1.0299311705981942,ps_car_11_cat_1.0599545151802927,ps_car_11_cat_1.0899778597623915,ps_car_11_cat_1.1200012043444902,ps_car_11_cat_1.1500245489265888,ps_car_11_cat_1.1800478935086876,ps_car_11_cat_1.210071238090786,ps_car_11_cat_1.2400945826728849
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,2.000000e+05,2.000000e+05,2.000000e+05,...,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,-4.661160e-17,1.601563e-16,-4.462208e-17,4.348522e-17,6.785683e-17,-1.007550e-16,0.000000,2.046363e-17,-4.220624e-17,-6.252776e-18,...,0.005090,0.003105,0.00319,0.004255,0.016775,0.009200,0.012870,0.003385,0.035640,0.159235
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,1.000003e+00,1.000003e+00,1.000003e+00,...,0.071163,0.055636,0.05639,0.065092,0.128428,0.095475,0.112714,0.058082,0.185391,0.365896
min,-9.905914e-01,-1.636898e+00,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.905914e-01,-9.067708e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-4.948532e-01,-1.766434e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.966230e-01,9.185477e-01,1.359482e+00,1.544515e+00,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.479576e+00,2.378802e+00,1.359482e+00,1.544515e+00,2.164348e+00,2.151853e+00,46.115719,2.277017e+01,9.200712e+00,3.027261e+01,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
printScore(getScore(X_train, y_train, X_test, y_test))

Accuracy: 0.5957
Precesion: 0.6022
Recall: 0.5643
F1 Score: 0.5826


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [36]:
CTR_data = data.copy()

categories = []

for header in data:
    if "cat" in header:
        categories.append(header)
        unique = np.unique(data[header])
        for value in unique:
            CTR_data[header][data[header] == value] = np.mean(target[data[header] == value])

X_train, X_test, y_train, y_test = train_test_split(normalize(CTR_data), target, test_size=0.5)
CTR_data.describe()

C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\4549987.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  CTR_data[header][data[header] == value] = np.mean(target[data[header] == value])
C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\454

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500000,4.483870,0.500000,0.500000,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015959,2.739255,0.020419,0.050252,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.494915,2.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.494915,4.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.500783,7.000000,0.517318,0.480547,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.911215,11.000000,0.964072,0.697423,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
printScore(getScore(X_train, y_train, X_test, y_test))

Accuracy: 0.5924
Precesion: 0.6008
Recall: 0.5597
F1 Score: 0.5796


__Вывод:__

Качество
Качество между моделью обученной с использованием счетчиков (CTR) и OneHotEncoding не так сильно различается. Значения упали всего на ~0.15-0.3%, что не является критическим.

Время обучения
Время в свою очередь сильно изменилось. С целых 2 секунд до жалких 0.3 секунд. Время улучшилось в 6 раз

Итог
По сравнению с выигрышем времени проигрыш в точности никчемен, поэтому CTR показывает свое превосходство в данном случае над OneHotEncoding

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:

вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [38]:
CTR_noise_data = data.copy()

for header in categories:
    uniq = np.unique(data[header])
    for value in uniq:
        current = np.array(target[data[header] == value])
        CTR_noise_data[header][data[header] == value] = np.mean(current) + np.random.random(current.shape) * 0.02

X_train, X_test, y_train, y_test = train_test_split(normalize(CTR_noise_data), target, test_size=0.5)
CTR_noise_data.describe()

C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\231768220.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  CTR_noise_data[header][data[header] == value] = np.mean(current) + np.random.random(current.shape) * 0.02
C:\Users\Егор\AppData\Loc

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.510003,4.483870,0.510025,0.510014,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.016971,2.739255,0.021243,0.050589,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.501772,2.000000,0.494998,0.486390,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.508342,4.000000,0.503814,0.492237,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.514129,7.000000,0.525819,0.498046,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.931161,11.000000,0.984059,0.717413,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
printScore(getScore(X_train, y_train, X_test, y_test))

Accuracy: 0.5930
Precesion: 0.5963
Recall: 0.5682
F1 Score: 0.5820


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def minim(a, b):
    if a < b:
        return a
    return b

fold_data = data.copy()

# небольшая вспомогательная функция
def get_fold_mean(idx, cumsum, fold_size, n):
    fold_num = idx // fold_size
    l = fold_num * fold_size
    r = minim(n, (fold_num + 1) * fold_size)
    total = cumsum[-1]  # полная сумма по всей группе
    fold_sum = cumsum[r] - cumsum[l]
    out_of_fold_sum = total - fold_sum
    out_of_fold_count = n - (r - l)
    if out_of_fold_count == 0:
        return total / n  # если нечего оставить, берем общее среднее
    return out_of_fold_sum / out_of_fold_count

cnt_blocks = 8

for v in categories:
    ans = []
    uniq = np.unique(data[v])
    cumsum = dict()
    cnt = dict()

    for u in uniq:
        np_data = np.array(target[data[v] == u])
        cumsum[u] = np.hstack(([0], np.cumsum(np_data)))

    for value in data[v]:
        if value not in cnt:
            cnt[value] = 0
        n = len(cumsum[value]) - 1
        fold_size = (n + cnt_blocks - 1) // cnt_blocks  # чтобы равномерно делить
        ans.append(get_fold_mean(cnt[value], cumsum[value], fold_size, n))
        cnt[value] += 1

    fold_data[v] = ans  # порядок уже совпадает
# дальше нормализуем и делим
X_train, X_test, y_train, y_test = train_test_split(normalize(fold_data), target, test_size=0.5)

fold_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.499997,4.483870,0.499997,0.499984,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.072264,2.739255,0.072364,0.085039,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.422759,0.000000,0.412820,0.406339,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.422759,2.000000,0.428571,0.406339,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.565613,4.000000,0.555677,0.549196,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.565618,7.000000,0.555677,0.549196,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,1.000000,11.000000,1.000000,0.797091,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [41]:
printScore(getScore(X_train, y_train, X_test, y_test))

Accuracy: 0.9997
Precesion: 0.9997
Recall: 0.9997
F1 Score: 0.9997


__Вывод:__